# Server

In [1]:
import os
import pickle
import socket
import struct
import time
from pathlib import Path
from typing import List, Union, Tuple
import math

import h5py
import numpy as np
import tenseal as ts
import torch
import torch.nn as nn
from torch import Tensor
from icecream import ic
ic.configureOutput(includeContext=True)
from tenseal.enc_context import Context
from tenseal.tensors.ckkstensor import CKKSTensor
from tenseal.tensors.ckksvector import CKKSVector
from tenseal.tensors.plaintensor import PlainTensor
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader, Dataset
print(f'tenseal version: {ts.__version__}')
print(f'torch version: {torch.__version__}')

tenseal version: 0.3.5
torch version: 1.8.1+cu102


## Establish connection with the client

In [2]:
host = 'localhost'
port = 10080
max_recv = 4096

In [3]:
# def init_socket(host, port):
#     """[summary]

#     Args:
#         host ([str]): [description]
#         port ([int]): [description]
#     """
#     sock = socket.socket()
#     sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
#     sock.bind((host, port))  # associates the socket with its local address
#     sock.listen()
#     print('Listening on', (host, port))
#     conn, addr = sock.accept()  # wait for the client to connect
#     print(f'Connection: {conn} \nAddress: {addr}')
#     return sock


s = socket.socket()
s.connect((host, port))

Listening on ('localhost', 10080)
Connection: <socket.socket fd=57, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 10080), raddr=('127.0.0.1', 35466)> 
Address: ('127.0.0.1', 35466)


Functions needed to send and receive messages in byte streams with the client

In [4]:
def send_msg(sock, msg) -> None:
    """Function used to send the message via socket
    """
    # prefix each message with a 4-byte length in network byte order
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    return recvall(sock, msglen)

## Hyperparameters

In [5]:
# dry_run = 0
# if dry_run:
#     batch_size = 2
#     epoch = 2
#     total_batch = 2
# else:
#     batch_size = 4
#     epoch = 400
#     total_batch = 414
#     # total_batch = 10
# lr = 0.001
# seed = 0

# hyperparams = {
#     'dry_run': dry_run,
#     'batch_size': batch_size,
#     'total_batch': total_batch,
#     'epoch': epoch,
#     'lr': lr,
#     'seed': seed
# }

Send the hyperparameters to the Client

In [5]:
recv_msg(sock)

OSError: [Errno 107] Transport endpoint is not connected